In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F
import torch.utils.data as data_utils
import matplotlib.pyplot as plt
from torchdiffeq import odeint
from torchcontrol.arch_cpugpu import HDNN, HDNN_Observer
from torchcontrol.utils import genpoints
import scipy
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from torch.nn import LSTM
from matplotlib2tikz import save as tikz_save
import time as time

In [ ]:
device = torch.device('cpu')
if torch.cuda.is_available(): device = torch.device('cuda')
print(device)

# Duffin Oscillator

$$\begin{bmatrix}\dot{u}\\\dot{v}\end{bmatrix}= \begin{bmatrix}v\\-\alpha u - k v -\beta u^3\end{bmatrix}$$
$$\Leftrightarrow\dot{x}= \Psi(x)$$

In [ ]:
a = 1.0
b = 1
c = 1.0
d = 1
# Dissipation rate
k = 0.5

In [ ]:
# define the ODE
def LV(x,t):
    u = x[0]
    v = x[1]
    dudt = v#a*u - b*u*v #
    dvdt = -a*u - k*v -b*pow(u,3) #-c*v +d*u*v + k*(a-b*v)*v#
    return [dudt,dvdt]

In [ ]:
# Simulate the systems
N = 200
Tf = 4
Ts = Tf/N
print('The sampling time is ',Ts,'s')
t = np.linspace(0,Tf,N)
# standard form
x0 = [1.5,1]
x = odeint(LV,x0,t)
x1 = odeint(LV,x[-1],t)
t = np.hstack((t,t+t[-1]))
x = np.vstack((x,x1))

In [ ]:
# save traj
orig_traj = x

In [ ]:
Np = 20
xlim = np.hstack((orig_traj.min(0)[0],orig_traj.max(0)[0]))
ylim = np.hstack((orig_traj.min(0)[1],orig_traj.max(0)[1]))

x1 = np.linspace(xlim[0],xlim[1],Np)
x2 = np.linspace(ylim[0],ylim[1],Np)
X1,X2 = np.meshgrid(x1,x2)

U = np.zeros((Np,Np))
V = np.zeros((Np,Np))

for i in range(Np):
    for j in range(Np):
        U[i,j], V[i,j] = LV([X1[i,j],X2[i,j]],0)

fig, ax = plt.subplots()
q = ax.quiver(X1, X2, U, V,scale = 80)
ax.plot(x[:,0],x[:,1])
ax.scatter(x[:,0],x[:,1], color = 'r')
plt.xlim([xlim[0],xlim[1]])
plt.ylim([ylim[0],ylim[1]])


X1a = X1.reshape(int(Np*Np),1)
X2a = X2.reshape(int(Np*Np),1)
Ua = U.reshape(int(Np*Np),1)
Va = V.reshape(int(Np*Np),1)

VecField = np.hstack((X1a,np.hstack((X2a,np.hstack((Ua,Va))))))
np.savetxt('VecField.dat', VecField, fmt=['%.3f','%.3f','%.3f','%.3f'],delimiter='\t',header="x\ty\tu\tv",comments = '')

In [ ]:
fig, ax = plt.subplots()
ax.plot(x[:,0],x[:,1])
ax.scatter(x[:,0],x[:,1], color = 'r')
plt.xlim([xlim[0],xlim[1]])
plt.ylim([ylim[0],ylim[1]])
tikz_save('Traj.tex')


## HDNN

Two methods attempted: 1st - approximation of derivative with $$\hat{y}_i = \frac{(\hat{u}_{i+1} - \hat{u}_{i})}{Ts}$$
2nd - use as labels $$LV(x_{i}, t)$$

In [ ]:
x = torch.Tensor(x).to(device)
x[1:]
x0 = x[0].view(1,2)

In [ ]:
# with approx
grad = ([(x[i+1] - x[i])/Ts for i in range(len(x)-1)])
grad = torch.Tensor([g.cpu().numpy() for g in grad])
# with real LV
#grad = ([LV(x[i], t) for i in range(len(x)-1)])
#grad = torch.Tensor([g for g in grad])

In [ ]:
train = data_utils.TensorDataset(x[:-1], grad)
trainloader = data_utils.DataLoader(train, batch_size=len(orig_traj)-1)

In [ ]:
# Define Neural Network
obs = HDNN('MLP', [[2, 16, 16, 2], False], [1,1,0], 0.5, odeint='cpu')

In [ ]:
X1t = torch.Tensor(X1).to(device)
X2t = torch.Tensor(X2).to(device)
Up = np.zeros((Np,Np))
Vp = np.zeros((Np,Np))

for i in range(Np):
    for j in range(Np):
        point = torch.Tensor([X1[i,j],X2[i,j]]).to(device)
        Up[i,j], Vp[i,j] = obs.predictor(point.view(1,2))[0][0],obs.predictor(point.view(1,2))[0][1]

Uo, Vo = Up, Vp

fig, ax = plt.subplots()
q = ax.quiver(X1, X2, Up, Vp,scale = 80)
plt.title("Pre training approximated vector field")

In [ ]:
epoch = 200000
ode_t = 0.001

In [ ]:
start_time = time.time()
obs.fit(trainloader, epoch=200000, time_delta=100, iter_accuracy=float('inf'), ode_t=0.001, ode_step=2, criterion='mse')
elapsed_time = time.time() - start_time

In [ ]:
print(elapsed_time)

In [ ]:
# Plot Loss function
#obs.plotLoss()
J = ([np.array(obs.pLoss[i].to('cpu').detach().numpy(),dtype='float').tolist() for i in range(len(obs.pLoss))])

T = np.linspace(0,epoch*ode_t,len(J))
plt.figure()
plt.plot(T[0:-1:10],J[0:-1:10])
plt.xlabel("$t$")
plt.ylabel("$\mathcal{J}^*(t)$")
plt.xlim([0,30])

plt.savefig('LossVF.eps', format='eps', dpi=300)
tikz_save('LossVF.tex')

In [ ]:
p = len(obs.pWdot)
theta = np.zeros((len(J),p))
omega = np.zeros((len(J),p))
for j in range(p):
    theta[:,j] = ([obs.pW[j][i].tolist() for i in range(len(J))])
    omega[:,j] = ([obs.pWdot[j][i].tolist() for i in range(len(J))])
    

In [ ]:
# from matplotlib import rc
# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
# ## for Palatino and other serif fonts use:
# #rc('font',**{'family':'serif','serif':['Palatino']})
# rc('text', usetex=True)
colors = plt.cm.jet(np.linspace(0,1,len(theta)))
plt.figure(figsize=(7,12))
plt.subplot(311);
skip = 20
#for i in range(p-300):
#    plt.plot(T, theta[:,i], color=colors[i])
plt.plot(T[0:-1:skip],theta[0:-1:skip], Linewidth = 3, alpha = 1);
#plt.plot(T,theta[:,0:47],Linewidth = 3, color = 'b',alpha = 0.5);
#plt.plot(T,theta[:,48:48+272],Linewidth = 3, color = 'r',alpha = 0.5);
#plt.plot(T,theta[:,48+273:-1],Linewidth = 3, color = 'g',alpha = 0.5);
#plt.xlabel("$t$");
plt.ylabel("$\theta$");
plt.xlim([0,30])
plt.subplot(312);
plt.plot(T[0:-1:skip],omega[0:-1:skip],Linewidth = 3);
#plt.xlabel("$t$");
plt.ylabel("$\omega(t)$");
plt.xlim([0,30])
plt.subplot(313);
plt.plot(T[0:-1:skip],J[0:-1:skip],Linewidth = 3);
#plt.xlabel("$t$");
plt.ylabel("$\mathcal{J}^*(t)$")
plt.xlim([0,30])
#plt.savefig('weightsVF.eps', format='eps', dpi=300);
tikz_save('weightsVF.tex');

### Compute reconstructed vector field & reconstruction error

given the linear system:
\begin{equation}
    \dot{x} = \Psi(x)
\end{equation}
Let the neural network model be
\begin{equation}
    y = f(x,\vartheta)
\end{equation}
After the training, the reconstruction error is computed (in a certain quantized region of the state-space) as
\begin{equation}
    E(x) = \|\Psi(x)-f(x,\vartheta^*)\|_2^2
\end{equation}
where $\vartheta^*$ is the trained vector of parameters

In [ ]:
Up = np.zeros((Np,Np))
Vp = np.zeros((Np,Np))

for i in range(int(Np)):
    for j in range(Np):
        point = torch.Tensor([X1[i,j],X2[i,j]]).to(device)
        Up[i,j], Vp[i,j] = obs.predictor(point.view(1,2))[0][0], obs.predictor(point.view(1,2))[0][1]

fig, ax = plt.subplots(figsize=(10,7))
q = ax.quiver(X1, X2, U, V,scale = 80)
q = ax.quiver(X1, X2, Up, Vp, scale = 80,color = 'r')
plt.title("Learned vec field vs ground truth")

Upa = Up.reshape(int(Np*Np),1)
Vpa = Vp.reshape(int(Np*Np),1)

RecVecField = np.hstack((X1a,np.hstack((X2a,np.hstack((Upa,Vpa))))))
np.savetxt('RecVecField.dat', RecVecField, fmt=['%.3f','%.3f','%.3f','%.3f'],delimiter='\t',header="x\ty\tu\tv",comments = '')

## Error contour plot

In [ ]:
E = np.zeros((Np,Np))

for i in range(Np):
    for j in range(Np):
        E[i,j] = np.sqrt(pow(U[i,j]-Up[i,j],2)+pow(V[i,j]-Vp[i,j],2))#/np.sqrt(pow(U[i,j],2)+pow(V[i,j],2))

In [ ]:

#### Plot contour map of the vector field's reconstruction error
#plt.figure(figsize=(12,8))
plt.contourf(X1, X2, E,100,cmap='Blues')
plt.plot(orig_traj[:,0],orig_traj[:,1], color='black')
plt.colorbar()
plt.title("Vector Field reconstruction Error")

Ea = E.reshape(int(Np*Np),1)

Error = np.hstack((X1a,np.hstack((X2a,Ea))))
np.savetxt('Error.dat', Error, fmt=['%.3f','%.3f','%.3f'])

## Reconstructed trajectory

We want to compare the reference trajectory $x_{r}(t)$ obtained by integrating the ODE $\dot{x}=\Psi(x)$ with $x_r(0) = x_0$ with a trajectory $\hat{x}(t)$ obtained by integrating $\dot{x}= f(x,\theta^*)$ (the NN model) with the same initial condition.

We can then evaluate how the reconstructed trajectory remains close to $x_r(t)$

In [ ]:
def LV_Learned(xi0, t):
    return obs.predictor(torch.Tensor(xi0).view(1,2).to(device)).flatten().detach().cpu()

In [ ]:
func = LV_Learned
t = np.linspace(0,2*Tf,2*N)
x0 = [1.5, 1]
sol = odeint(func, x0, t)

In [ ]:
plt.figure(figsize=(10,7))
plt.plot([s[0] for s in sol], [s[1] for s in sol])
plt.plot(orig_traj[:,0],orig_traj[:,1], color='black')
plt.legend(['Learned','Original'])

In [ ]:
e = ([np.linalg.norm(sol[i]-orig_traj[i]) for i in range(len(orig_traj))])

plt.figure()
plt.plot(e)